# Google Colab - Multiple neural network analysis

<sup>This notebook is a part of Natural Language Processing class at the University of Ljubljana, Faculty for computer and information science. Please contact [slavko.zitnik@fri.uni-lj.si](mailto:slavko.zitnik@fri.uni-lj.si) for any comments.</sub>

## Google Colab specifics

Before running this notebook you should select GPU-accelerated environment.

### Libraries installation

Some of the libraries are already available but you should manually install them:

In [ ]:
!pip install tensorflow-gpu keras pandas numpy

### I/O device registering

Current working directory is set to `/content` by default. You can also give access to your Google Drive to save models/results/... there.

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

# Access your Drive data using folder '/content/drive/MyDrive'

### GPU device selection review

You can directly use system command `nvidia-smi` or use Python library (e.g. Tensorflow or PyTorch) to check this.

In [ ]:
!nvidia-smi

In [ ]:
import tensorflow as tf
import os
print(f"Tensorflow version: {tf.__version__}")

# Restrict TensorFlow to only allocate 4GBs of memory on the first GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(f"The system contains '{len(gpus)}' Physical GPUs and '{len(logical_gpus)}' Logical GPUs")
  except RuntimeError as e:
    print(e)
else:
    print(f"Your system does not contain a GPU that could be used by Tensorflow!")

## IMDB sentiment analysis example

First we download the IMDB dataset. We present each word with a specific index from a vocabulary of 10000 words.

In [1]:
import numpy as np
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.preprocessing.text import text_to_word_sequence

# number of distinct words
vocabulary_size = 10000

# number of words per review
max_review_length = 500

# load Keras IMDB movie reviews dataset
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocabulary_size)

print(f'Number of reviews: {len(X_train)}.')
print(f'First review: \n\t{X_train[0]}.')
print(f'First label: {y_train[0]}.')
print(f'Length of first review before padding: {len(X_train[0])}.')

# padding reviews
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
print(f"\nAfter padding:")
print(f'First review: \n\t{X_train[0]}.')
print(f'Length of first review after padding: {len(X_train[0])}.')

Using TensorFlow backend.
/Users/slavkoz/opt/anaconda3/envs/nlp-course-fri/lib/python3.6/site-packages/keras/datasets/imdb.py:101: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/Users/slavkoz/opt/anaconda3/envs/nlp-course-fri/lib/python3.6/site-packages/keras/datasets/imdb.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


Number of reviews: 25000.
First review: 
	[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32].
First label: 1.
L

Mapping between real words and indexes:

In [2]:
# A dictionary mapping words to an integer index
word_index = imdb.get_word_index()

# The first indices are reserved, so we map the index for our use
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

# Decode review text
def decode_review(text_ids, cls):
    text = ' '.join([reverse_word_index.get(i, '?') for i in text_ids if i not in [0,1,2,3]])
    label = 'POSITIVE' if cls == 1 else 'NEGATIVE'
    return f"\tText: {text}\n\tLabel: {label}"

# First review
print(f"First review: \n{decode_review(X_test[0], y_test[0])}")
# Last review
print(f"\nLast review: \n{decode_review(X_test[len(X_test)-1], y_test[len(X_test)-1])}")

First review: 
	Text: please give this one a miss br br and the rest of the cast rendered terrible performances the show is flat flat flat br br i don't know how michael madison could have allowed this one on his plate he almost seemed to know this wasn't going to work out and his performance was quite so all you madison fans give this a miss
	Label: NEGATIVE

Last review: 
	Text: a good ol' boy film is almost required to have car chases a storyline that has a vague resemblance to plot and at least one very pretty country gal with short shorts and a low top the pretty gal is here dressed in designer but the redneck stop there jimmy dean is a natural as a but as a tough guy former sheriff he comes up way short big john is big but he isn't convincing with the bad part of his bug eyed jack is a hoot as always and bo hopkins has been playing this same part for decades ned beatty also does his part in a small role but there is no story it more like an episode of in the heat of the night tha

Below we create multiple models and evaluate their performance:

* **FFN**: Input to the models are word indices directly fed into a Dense layer.
* **FFN with embeddings**: After creation of embedding vectors, the same architecture as in *FFN* is used.
* **CNN**: Similar to the *FFN with embeddings* model with a convoluational and pooling layer immediatelly after embedding layer,
* **RNN**: Simple RNN model with 100 hidden dimensions and prediction at the end.
* **CNN+RNN**: A combination of *CNN* and *RNN* models above.

The runtime for the below script should take about a half hour using Tesla V100 32GB GPU.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv1D, MaxPooling1D
import pandas as pd

EMBEDDING_DIM = 256

# Fully connected neural network
model_ffn = Sequential()
model_ffn.add(Dense(250, activation='relu',input_dim=max_review_length))
model_ffn.add(Dense(1, activation='sigmoid'))
model_ffn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history_ffn = model_ffn.fit(X_train, y_train, epochs=20, batch_size=128, verbose=2)
scores_ffn = model_ffn.evaluate(X_test, y_test, verbose=0)

# Fully connected neural network with embeddings
model_ffne = Sequential()
model_ffne.add(Embedding(vocabulary_size, EMBEDDING_DIM, input_length=max_review_length))
model_ffne.add(Flatten())
model_ffne.add(Dense(250, activation='relu'))
model_ffne.add(Dense(1, activation='sigmoid'))
model_ffne.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history_ffne = model_ffne.fit(X_train, y_train, epochs=20, batch_size=128, verbose=2)
scores_ffne = model_ffne.evaluate(X_test, y_test, verbose=0)

# Convolutional neural network
model_cnn = Sequential()
model_cnn.add(Embedding(vocabulary_size, EMBEDDING_DIM, input_length=max_review_length))
model_cnn.add(Conv1D(filters=200, kernel_size=3, padding='same', activation='relu'))
model_cnn.add(MaxPooling1D(pool_size=2))
model_cnn.add(Flatten())
model_cnn.add(Dense(250, activation='relu'))
model_cnn.add(Dense(1, activation='sigmoid'))
model_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history_cnn = model_cnn.fit(X_train, y_train, epochs=20, batch_size=128, verbose=2)
scores_cnn = model_cnn.evaluate(X_test, y_test, verbose=0)

# Recurrent Neural Network
model_rnn = Sequential()
model_rnn.add(Embedding(vocabulary_size, EMBEDDING_DIM, input_length=max_review_length))
model_rnn.add(SimpleRNN(100))
model_rnn.add(Dense(1, activation='sigmoid'))
model_rnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history_rnn = model_rnn.fit(X_train, y_train, epochs=3, batch_size=64)
scores_rnn = model_rnn.evaluate(X_test, y_test, verbose=0)

# Convolutional + Recurrent Neural Network
model_cnn_rnn = Sequential()
model_cnn_rnn.add(Embedding(vocabulary_size, EMBEDDING_DIM, input_length=max_review_length))
model_cnn_rnn.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model_cnn_rnn.add(MaxPooling1D(pool_size=2))
model_cnn_rnn.add(SimpleRNN(100))
model_cnn_rnn.add(Dense(1, activation='sigmoid'))
model_cnn_rnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history_cnn_rnn = model_cnn_rnn.fit(X_train, y_train, epochs=10, batch_size=64)
scores_cnn_rnn = model_cnn_rnn.evaluate(X_test, y_test, verbose=0)

# Evaluation
df = pd.DataFrame({'Model': ['FFNN', 'FFNN with Embeddings', 'CNN', 'RNN', 'CNN+RNN'],
                   'Accuracy': [str(round(scores_ffn[1]*100, 2)) + '%',
                                str(round(scores_ffne[1]*100, 2)) + '%',
                                str(round(scores_cnn[1]*100, 2)) + '%',
                                str(round(scores_rnn[1]*100, 2)) + '%',
                                str(round(scores_cnn_rnn[1]*100, 2)) + '%']})
print(df)

Epoch 1/20
 - 5s - loss: 183.3908 - accuracy: 0.5076
Epoch 2/20
 - 3s - loss: 50.0721 - accuracy: 0.5835
Epoch 3/20
 - 3s - loss: 18.2098 - accuracy: 0.6580
Epoch 4/20
 - 3s - loss: 7.4457 - accuracy: 0.7231
Epoch 5/20
 - 3s - loss: 3.6657 - accuracy: 0.7660
Epoch 6/20
 - 3s - loss: 2.0616 - accuracy: 0.7935
Epoch 7/20
 - 3s - loss: 1.4220 - accuracy: 0.8069
Epoch 8/20
 - 3s - loss: 0.9844 - accuracy: 0.8314
Epoch 9/20
 - 3s - loss: 0.8532 - accuracy: 0.8371
Epoch 10/20
 - 3s - loss: 0.7265 - accuracy: 0.8482
Epoch 11/20
 - 3s - loss: 0.6246 - accuracy: 0.8623
Epoch 12/20
 - 3s - loss: 0.5957 - accuracy: 0.8682
Epoch 13/20
 - 3s - loss: 0.5488 - accuracy: 0.8803
Epoch 14/20
 - 3s - loss: 0.4813 - accuracy: 0.8908
Epoch 15/20
 - 4s - loss: 0.5251 - accuracy: 0.8903
Epoch 16/20
 - 3s - loss: 0.6005 - accuracy: 0.8911
Epoch 17/20
 - 3s - loss: 0.6045 - accuracy: 0.8936
Epoch 18/20
 - 2s - loss: 0.6383 - accuracy: 0.8940
Epoch 19/20
 - 3s - loss: 0.6001 - accuracy: 0.8982
Epoch 20/20
 - 3s

/Users/slavkoz/opt/anaconda3/envs/nlp-course-fri/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/20
 - 286s - loss: 0.5827 - accuracy: 0.7016
Epoch 2/20
 - 282s - loss: 0.1630 - accuracy: 0.9381
Epoch 3/20
 - 286s - loss: 0.0239 - accuracy: 0.9962
Epoch 4/20
 - 272s - loss: 0.0029 - accuracy: 0.9999
Epoch 5/20
 - 283s - loss: 6.1124e-04 - accuracy: 1.0000
Epoch 6/20
 - 308s - loss: 2.9286e-04 - accuracy: 1.0000
Epoch 7/20
 - 325s - loss: 1.8083e-04 - accuracy: 1.0000
Epoch 8/20
 - 286s - loss: 1.2173e-04 - accuracy: 1.0000
Epoch 9/20
 - 270s - loss: 8.7555e-05 - accuracy: 1.0000
Epoch 10/20
 - 272s - loss: 6.4893e-05 - accuracy: 1.0000
Epoch 11/20
 - 271s - loss: 4.9991e-05 - accuracy: 1.0000
Epoch 12/20
 - 274s - loss: 3.9237e-05 - accuracy: 1.0000
Epoch 13/20
 - 311s - loss: 3.1467e-05 - accuracy: 1.0000
Epoch 14/20


The code above should output something similar to the following:

```
                            Model Accuracy
          0                  FFNN   50.79%
          1  FFNN with Embeddings   87.64%
          2                   CNN   87.55%
          3                   RNN   61.14%
          4               CNN+RNN   82.62%
```

Surprisingly, the fully connected neural network with embeddings and convolutional neural network outperform the remaining networks. FFN is a very simple network with a single 250 dimensional dense layer.

CNNs can be robust to the position and orientation of learned objects in the image, while the same principle can be used on sequences, such as the one-dimensional sequence of words in a movie review. A simple RNN seems not to be very competitive but together with a CNN it achieves a decent performance.

Now, let's test the models with some custom examples:

In [ ]:
def movie_sentiment(reviews, 
                    models=[model_ffn, model_ffne, model_cnn, model_rnn, model_cnn_rnn],
                    titles=['FFN', 'FFNE', 'CNN', 'RNN', 'CNN+RNN']):
    df = pd.DataFrame(columns=['review'] + titles)
    i = 0
    for review in reviews:
        words = set(text_to_word_sequence(review))
        words = [word_index[w] for w in words]
        words = sequence.pad_sequences([words], maxlen=max_review_length)
        df.loc[i] = [review] + titles
        df.loc[i]['review'] = review
        for j, model in enumerate(models):
            proba = model.predict(words)
            sentiment = '+' if proba>0.5 else '-'
            df.loc[i][titles[j]] = sentiment
        i = i + 1
    return df
    
text1 = 'I like it'
text2 = 'I dont like it'
text3 = 'After 30 min I still did not know what the movie is about'
text4 = 'It is so good that I will never ever watch it again. Boring experience.'
text5 = "It is like the Titanic movie!"
text6 = "That is the best movie I have ever seen."
text7 = "That is the worst movie I have ever seen."
print(movie_sentiment([text1, text2, text3, text4, text5, text6, text7]))

The test above should output something similar to the following:

```
                                              review FFN FFNE CNN RNN CNN+RNN  | EXPECTED
0                                          I like it   -    +   +   +       -  |        + 
1                                     I dont like it   +    -   +   +       -  |        -
2  After 30 min I still did not know what the mov...   +    +   +   -       -  |        -
3  It is so good that I will never ever watch it ...   -    +   +   -       -  |        -
4                      It is like the Titanic movie!   +    +   +   +       -  |        +
5           That is the best movie I have ever seen.   +    +   +   +       -  |        +
6          That is the worst movie I have ever seen.   -    -   -   -       -  |        -
-----------------------------------------------------------------------------------------
                                 CORRECT PREDICTIONS   4    5   4   6       4  |        7
```

It seems that RNN performed as the best on these few short examples.

Try different architectures, models, hyperparameters (e.g. embedding dimensions), ... and you might improve the results.

## References

* The example above follows the post by Michel Kana, PhD: [Sentiment analysis: a practical benchmark](https://towardsdatascience.com/sentiment-analysis-a-benchmark-903279cab44a)